In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(sdk.EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "BrownianMotion", "refs": [], "args": [0.2, 2], "start": 0.1}], "evaluations": [{"state": 0, "time": 2, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 1, "MemoryLimitKB": 1}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.49474653601646423],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'run_time_seconds': {'mean': 0.00032092301989905536,
  'n': 512,
  'skewness': None,
  'stddev': 7.638641932317114e-07},
 'skewness': [0.0018847952596843243],
 'stddev': [2.8238706588745117],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(sdk.EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.2993745505809784,
  0.4962975084781647,
  1.0968090295791626,
  2.102588653564453],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'run_time_seconds': {'mean': 2.0929782390594482,
  'n': 512,
  'skewness': None,
  'stddev': None},
 'skewness': [0.002113355090841651,
  -0.0010458247270435095,
  -0.001311505096964538,
  0.002931001130491495],
 'stddev': [2.001894474029541,
  2.831454277038574,
  4.4775872230529785,
  6.3296685218811035],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [100, 200, 500, 1000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

0.5 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(sdk.IndependentGaussian())
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(sdk.EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.31300678849220276,
  0.3037349581718445,
  0.48782822489738464,
  0.4769749939441681,
  1.123900055885315,
  1.0785328149795532],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'run_time_seconds': {'mean': 0.33200767636299133,
  'n': 512,
  'skewness': None,
  'stddev': 6.0673777625197545e-05},
 'skewness': [0.0007159485830925405,
  0.018429366871714592,
  -0.002158256247639656,
  0.021832061931490898,
  -0.0398055762052536,
  -0.02540764957666397],
 'stddev': [2.00711989402771,
  2.0029661655426025,
  2.830491065979004,
  2.828368663787842,
  4.474641799926758,
  4.4463210105896],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
response = requests.post(f'{server}/model',model.json())
results = sdk.EvaluationResults_from_json_string(response.text)
results.model = model

In [10]:
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.292817,0.019909,1.990857,0.031832
1,BrownianMotion,,1,1.0,2000,10000,0.321392,0.019882,1.988194,0.014055
2,BrownianMotion,,0,2.0,4000,10000,0.462984,0.028155,2.815451,0.020987
3,BrownianMotion,,1,2.0,4000,10000,0.485998,0.027930,2.792999,0.016728
4,BrownianMotion,,0,5.0,10000,10000,1.057207,0.044456,4.445595,0.029747
5,BrownianMotion,,1,5.0,10000,10000,1.116393,0.044519,4.451866,-0.003743
